In [46]:
import csv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Read blackbox call history

In [47]:
column_names = list()
data = list()
is_first_row = True
with open('result_optimization_surrogate.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        if is_first_row:
            for x in row:
                column_names.append(x)
            is_first_row = False
        else:
            data.append([float(x) for x in row])
data = np.array(data)

sur_data = data[-100:, :]
x_train = sur_data[:, :6]
y_train = sur_data[:, 6]

# GP Approximation with noise

In [55]:
from sklearn import gaussian_process
#from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel#, Matern
import random

kernel1 = ConstantKernel() + RBF() + WhiteKernel(noise_level=1)
#kernel2 = ConstantKernel() + Matern(length_scale=2, nu=3/2) + WhiteKernel(noise_level=1)

y_noised = y_train

for i in range(100):
    y_noised[i] += random.gauss(0, 0.02*y_noised[i])

gp = gaussian_process.GaussianProcessRegressor(kernel=kernel1)

#from pySOT.surrogate import GPRegressor
#lb = np.array([3, 850, 850, 53, 1700000, 136])
#ub = np.array([5, 1150, 1150, 73, 2300000, 184])
#dim=6
#gp = GPRegressor(dim=dim, lb=lb, ub=ub)

gp.fit(x_train, y_noised)

GaussianProcessRegressor(kernel=1**2 + RBF(length_scale=1) + WhiteKernel(noise_level=1))

# Artificial_blackbox

In [56]:
def artificial_blackbox(args):
    x_pred = args.reshape(1,6)
    y_pred = gp.predict(x_pred, return_std=False)
    
    return y_pred

In [57]:
args = np.array([4, 1000, 1000, 63, 2000000, 160])
print(artificial_blackbox(args))

[30.53469073]
